In [ ]:
import meshplot as mp
import polyfempy as pf
import polyscope as ps
import numpy as np
import trimesh
import os, sys
from pprint import pprint as pprint

class Object3d:
    def __init__(self, vertices, faces):
        self.vertices = vertices
        self.faces = faces

# Load the mesh
plane_hole_mesh = trimesh.load("plane_hole.obj")


# Extract vertices and faces
plane_hole = Object3d(plane_hole_mesh.vertices, plane_hole_mesh.faces)


mp.plot(plane_hole.vertices, plane_hole.faces)


In [ ]:
def create_circle(radius=1, num_segments=30):
    # Create the circle
    angles = np.linspace(0, 2 * np.pi, num_segments, endpoint=False)
    x = radius * np.cos(angles)
    y = radius * np.sin(angles)
    vertices = np.vstack((x, y)).T

    # Create the faces (triangles) for the circle
    faces = []
    for i in range(num_segments):
        faces.append([i, (i + 1) % num_segments, num_segments])
    faces = np.array(faces)

    # Add center vertex
    center_vertex = np.array([[0, 0]])
    vertices = np.vstack((vertices, center_vertex))
    
    return Object3d(vertices, faces)


def get_edges(vertices, faces):
    edges = set()
    for face in faces:
        for i in range(len(face)):
            edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
            edges.add(edge)
    edge_vertices = np.array(list(edges))
    return edge_vertices


# Create the circle mesh
circle = create_circle(radius=1, num_segments=30)

# Create a Trimesh object
circle_mesh= trimesh.Trimesh(vertices=circle.vertices, faces=circle.faces)

# Save the mesh as an .obj file
circle_mesh.export('circle_mesh.obj')

# Plot the mesh
mp.plot(circle.vertices, circle.faces)



In [ ]:
def get_edges(vertices, faces):
    edges = set()
    for face in faces:
        for i in range(len(face)):
            edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
            edges.add(edge)
    return np.array(list(edges))


# Extract edges
edges = get_edges(circle.vertices, circle.faces)

# Prepare line start and end points
line_starts = circle.vertices[edges[:, 0]]
line_ends = circle.vertices[edges[:, 1]]

# Plot the mesh
p = mp.plot(circle.vertices, circle_mesh.faces, return_plot=True)

# Add the mesh borders
p.add_lines(line_starts, line_ends)


boundary_box = False
if boundary_box:
    # Get bounding box for the mesh
    min_coords = np.min(circle.vertices, axis=0)
    max_coords = np.max(circle.vertices, axis=0)

    # Corners of the bounding box
    v_box = np.array([[min_coords[0], min_coords[1], 0], [max_coords[0], min_coords[1], 0], 
                    [max_coords[0], max_coords[1], 0], [min_coords[0], max_coords[1], 0],
                    [min_coords[0], min_coords[1], 0], [max_coords[0], min_coords[1], 0], 
                    [max_coords[0], min_coords[1], 0], [max_coords[0], max_coords[1], 0], 
                    [min_coords[0], max_coords[1], 0], [min_coords[0], min_coords[1], 0]])

    # Edges of the bounding box
    f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0], 
                    [4, 5], [5, 6], [6, 7], [7, 4], 
                    [0, 4], [1, 5], [2, 6], [3, 7]])

    # Add bounding box edges
    p.add_edges(v_box, f_box, shading={"line_color": "red"})

    # Add bounding box vertices as points
    p.add_points(v_box, shading={"point_color": "green"})



In [ ]:
# Initialize polyscope
ps.init()

# Register the mesh with polyscope
ps.register_surface_mesh("circle", circle.vertices, circle.faces, smooth_shade=True)

# Optional: Add scalar or vector quantities if desired
# Example scalar function (just as a demo)
vertex_scalar = np.linspace(0, 1, len(circle.vertices))
ps.get_surface_mesh("circle").add_scalar_quantity("vertex_scalar", 
        vertex_scalar, defined_on='vertices', cmap='viridis')

# Example vector function (just as a demo)
# Since we only have 2D vertices, let's use simple vectors
face_vectors = np.zeros((len(circle.faces), 3))  # Zero vectors for demonstration
ps.get_surface_mesh("circle").add_vector_quantity("face_vectors", 
        face_vectors, defined_on='faces', color=(0.2, 0.5, 0.5))

# Show the 3D visualization
ps.show()

In [ ]:
settings = pf.Settings(
    discr_order=2,
    pde=pf.PDEs.LinearElasticity
)

settings.set_material_params("E", 2100)
settings.set_material_params("nu", 0.3)


problem = pf.Problem()

settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

#problem.set_x_symmetric(1)
#problem.set_y_symmetric(4)
problem.set_force(0, [100, 0])
settings.problem = problem

solver = pf.Solver()

solver.settings(settings)
solver.load_mesh_from_path("circle_mesh.obj", normalize_mesh=True)

solver.solve()

pts, tets, disp = solver.get_sampled_solution()

vertices = pts + disp

mises, _ = solver.get_sampled_mises_avg()

mp.plot(vertices, tets, mises, return_plot=True)

pprint(mises)

In [ ]:
help(problem.set_force)

In [ ]:
help(pf.PDEs)

In [ ]:
def create_triangulated_circle_mesh(radius=1, num_segments=30, num_radial=5):
    # Generate boundary vertices
    angles = np.linspace(0, 2 * np.pi, num_segments, endpoint=False)
    boundary_vertices = np.array([[radius * np.cos(angle), radius * np.sin(angle)] for angle in angles])

    # Generate radial vertices
    radial_vertices = []
    for i in range(1, num_radial):
        factor = i / num_radial
        radial_vertices.append(factor * radius * np.array([np.cos(angles), np.sin(angles)]).T)
    radial_vertices = np.vstack(radial_vertices)

    # Central vertex
    central_vertex = np.array([[0, 0]])

    # Combine all vertices
    vertices = np.vstack((central_vertex, boundary_vertices, radial_vertices))

    # Create faces
    faces = []
    # Add faces from the center to the boundary
    num_boundary = num_segments
    for i in range(num_boundary):
        next_i = (i + 1) % num_boundary
        for j in range(num_radial):
            radial_idx = num_boundary + j * num_boundary + i
            next_radial_idx = num_boundary + j * num_boundary + next_i
            if j == 0:
                faces.append([0, i + 1, next_i + 1])
            else:
                faces.append([radial_idx, next_radial_idx, i + 1])
                faces.append([radial_idx, next_i + 1, next_radial_idx])

    faces = np.array(faces)

    return vertices, faces

# Generate the circle mesh
vertices, faces = create_triangulated_circle_mesh(radius=1, num_segments=30, num_radial=5)

# Save to OBJ file
def save_obj(filename, vertices, faces):
    with open(filename, 'w') as file:
        for v in vertices:
            file.write(f"v {v[0]} {v[1]} 0.0\n")  # Assuming 2D mesh with z=0
        for face in faces:
            file.write(f"f {' '.join(str(f + 1) for f in face)}\n")  # OBJ format uses 1-based indexing

save_obj("triangulated_circle.obj", vertices, faces)

print(f"Mesh with {len(vertices)} vertices and {len(faces)} faces saved to 'triangulated_circle.obj'")



# Initialize polyscope
ps.init()

# Register the mesh with polyscope
ps.register_surface_mesh("circle", vertices, faces, smooth_shade=True)

# Optional: Add scalar or vector quantities if desired
# Example scalar function (just as a demo)
vertex_scalar = np.linspace(0, 1, len(vertices))
ps.get_surface_mesh("circle").add_scalar_quantity("vertex_scalar", 
        vertex_scalar, defined_on='vertices', cmap='viridis')

# Example vector function (just as a demo)
# Since we only have 2D vertices, let's use simple vectors
face_vectors = np.zeros((len(faces), 3))  # Zero vectors for demonstration
ps.get_surface_mesh("circle").add_vector_quantity("face_vectors", 
        face_vectors, defined_on='faces', color=(0.2, 0.5, 0.5))

# Show the 3D visualization
ps.show()